IMPORT LIBRAIRIES 

training fhmm

In [4]:
pip install collections

ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
import sys
import json 
import pandas as pd 
import numpy as np
import scipy as sp
from scipy import signal 
from hmmlearn.hmm import GaussianHMM
from collections import OrderedDict

train_power = OrderedDict()
data = pd.read_csv("./AMPds/ampds.csv")
train_power['BME'] = sp.signal.medfilt(data['BME'].values[0:20160]).reshape(-1,1)
train_power['DWE'] = sp.signal.medfilt(data['DWE'].values[0:20160]).reshape(-1,1)
train_power['FGE'] = sp.signal.medfilt(data['FGE'].values[0:20160]).reshape(-1,1)
train_power['HPE'] = sp.signal.medfilt(data['HPE'].values[0:20160]).reshape(-1,1)
train_power['UTE'] = sp.signal.medfilt(data['UTE'].values[0:10080]).reshape(-1,1)

num_appliances = len(train_power)


state_appliances = OrderedDict()
state_appliances['BME'] = 2
state_appliances['DWE'] = 3
state_appliances['FGE'] = 3
state_appliances['HPE'] = 2
state_appliances['UTE'] = 1


# Modeling each appliance with GaussianHMM
model = OrderedDict()
for appliance in train_power:
    model[appliance] = GaussianHMM(n_components=state_appliances[appliance], covariance_type = "full").fit(train_power[appliance])
    
startprob = OrderedDict()
for appliance in state_appliances:
    startprob[appliance] = np.array([1/state_appliances[appliance]]*state_appliances[appliance])
    

buff = "{"

for appliance in model: 
    buff += "\"" + appliance + "\" : {" + \
                    "\"startprob\" : " + str(startprob[appliance]) + \
                    ", \"transmat\" : " + str(model[appliance].transmat_.tolist()) + \
                    ", \"means\" : " + str(model[appliance].means_.tolist()) + \
                    ", \"covars\" : " + str(model[appliance].covars_.tolist()) + "},"
    
buff = buff[:-1]
buff += "}"

# Stocker les données entraînées dans un fichier texte
with open('trained_models.txt', 'w') as f:
    f.write(buff)

print(buff)

Model is not converging.  Current: 74031.27195902531 is not greater than 74031.27323383577. Delta is -0.00127481046365574


{"BME" : {"startprob" : [0.5 0.5], "transmat" : [[0.9871379900337447, 0.01286200996625539], [0.05240496348437618, 0.9475950365156238]], "means" : [[7.000000000000008], [291.9635202512493]], "covars" : [[[6.183605062374777e-07]], [[14571.35608579462]]]},"DWE" : {"startprob" : [0.33333333 0.33333333 0.33333333], "transmat" : [[0.9997435239420929, 0.0, 0.00025647605790717114], [0.0, 0.9208633092707765, 0.07913669072922358], [0.01295336616697602, 0.05699480281898657, 0.9300518310140374]], "means" : [[0.0], [752.9352517985611], [31.261649284048147]], "covars" : [[[5.129257311987589e-07]], [[223.8879299725903]], [[2679.7264214259744]]]},"FGE" : {"startprob" : [0.33333333 0.33333333 0.33333333], "transmat" : [[0.9200594353732741, 0.07994056462672591, 1.4855137090062693e-163], [0.04023332335880444, 0.9594675441254569, 0.0002991325157386258], [9.256678490088977e-157, 0.07017543859122632, 0.9298245614087737]], "means" : [[129.192867756315], [0.9092948478277125], [439.7894736842105]], "covars" : 

In [8]:
pip install ruptures 

     |████████████████████████████████| 1.2 MB 219 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


Testing fhmm

In [13]:
import sys
import json
import itertools
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from scipy import signal 
from copy import deepcopy
from hmmlearn.hmm import GaussianHMM
from collections import OrderedDict

""" --------------------- Function---------------------------"""

# Function to sort parameters from its lowest state
def return_sorting_mapping(means):
    means_copy = deepcopy(means)
    means_copy = np.sort(means_copy, axis=0)
    
    # Finding mapping
    mapping = {}
    for i, val in enumarate(means_copy):
        mapping[i] = np.where(val == means)[0][0]
    return mapping


def sort_startprob(mapping, startprob):
    num_elements = len(startprob)
    new_startprob = np.zeros(num_elements)
    for i in range(len(startprob)):
        new_startprob[i] = startprob[mapping[i]]
    return new_startprob


def sort_covars(mapping, covars):
    new_covars = np.zeros_like(covars)
    for i in range(len(covars)):
        new_covars[i] = covars[mapping[i]]
    return new_covars


def sort_transition_matrix(mapping, A):
    num_elements = len(A)
    A_new = np.zeros((new_elements, new_elements))
    for i in range(num_elements):
        for j in range(num_elements):
            A_new[i, j] = A[mapping[i], mapping[j]]
    return A_new

def sort_learnt_parameters(startprob, means, covars, transmat):
    mapping = return_sorting_mapping(means)
    means_new = np.sort(means, axis=0)
    startprob_new = sort_startprob(mapping,startprob)
    covars_new = sort_covars(mapping, covars)
    transmat_new = sort_transition_matrix(mapping, transmat)
    assert np.shape(means_new) == np.shape(means)
    assert np.shape(startprob_new) == np.shape(startprob)
    assert np.shape(transmat_new) == np.shape(transmat)
    
    return [startprob_new, means_new, covars_new, transmat_new]

# Function to combined each parameter of all appliances
def compute_A_fhmm(list_A):
    result = list_A[0]
    for i in range(len(list_A) -1):
        result = np.kron(result, list_A[i+1])
    return result

def compute_pi_fhmm(list_pi):
    result = list_pi[0]
    for i in range(len(list_pi)-1):
        result = np.kron(result, lis_pi[i+1])
    return result

def compute_means_fhmm(list_means):
    states_combination = list(itertools.product(*list_means))
    num_combination = list(states_combination)
    means_stacked = np.array([sum(x) for x in states_combination])
    means = np.reshape(means_stacked, (num_combination, 1))
    cov = np.tile(5*np.identity(1), (num_combinations, 1, 1))
    
    return [means, cov]

def create_combined_fhmm(model):
    list_pi = [model[appliance].startprob_ for appliance in model]
    list_A = [model[appliance].transmat_ for appliance in model]
    list_means = [model[appliance].means_.flatten().tolist()
                  for appliance in model]
    
    pi_combined = compute_pi_fhmm(list_pi)
    A_combined = computed_A_fhmm(list_A)
    [mean_combined, cov_combined] = computed_means_fhmm(list_means)
    
    combined_model = GaussianHMM(n_components = len(pi_combined), covariance_type = 'full')
    combined_model.startprob_ = pi_combined 
    combined_model.transmat_ = A_combined
    combined_model.means_ = means_combined 
    combined_model.covars_ = cov_combined
    
    return combined_model

# Disaggregate energy, finding state and power of each appliance 
def decode_hmm(length_sequence, centroids, appliance_list, states):
    hmm_states = OrderedDict()
    hmm_power = OrderedDict()
    total_num_combinations = 1
    
    for appliance in appliance_list: 
        total_num_combinations *= len(centroids[appliance])
      
    
    for appliance in appliance_list:
        hmm_states[appliance] = np.zeros(length_sequence, dtype = np.int)
        hmm_power[appliance] = np.zeros(length_sequence)
    
    
    for i in range(length_sequence):
        factor = total_num_combinations
        
        for appliance in appliance_list:
            factor = factor // len(centroids[appliance])
            temp = int(states[i])/factor
            hmm_states[appliance][i] = temp % len(centroids[appliance])
            hmm_power[appliance][i] = centroids[
                appliance][hmm_states[appliance][i]]
            
    return [hmm_states, hmm_power]

def disaggregate(combined_model, model, test_power):
    
    length = len(test_power)
    temp = test_power.reshape(length, 1)
    states = (combined_model.predict(temp))
    
    means = OrderedDict()
    for appliance in model: 
        means[appliance] = (
            model[appliance].means_.flatten().tolist())
        means[appliance].sort()
        
        
    [decoded_states, decoded_power] = decode_hmm(
        len(states), means, means.keys(), states)
    
    return [decode_power, decoded_states]

#data = sys.argv[1]
#data = json.loads(data)
data = pd.read_csv("./AMPds/test_ampds2.csv")

#model_appliance = data['model']

test_power = data["Total"].values

state_appliances = OrderedDict()
state_appliances['BME'] = 2
state_appliances['DWE'] = 3
state_appliances['FGE'] = 3
state_appliances['HPE'] = 2
state_appliances['UTE'] = 1

satrtprob = OrderedDict()
transmat = OrderedDict()
means = OrderedDict()
covars = OrderedDict()
model = OrderedDict()


for appliance in model_appliance: 
    startprob[appliance] = np.array(model_appliance[appliance]['startprob'])
    transmat[appliance] = np.array(model_appliance[appliance]['transmat'])
    means[appliance] = np.array(model_appliance[appliance]['means'])
    covars[appliance] = np.array(model_appliance[appliance]['covars'])
    
for appliance in model_appliance: 
    model[appliance] =  GaussianHMM(n_components=states_appliances[appliance], covariance_type ='full')
    model[appliance].startprob_ = startprob[appliance]
    model[appliance].transmat_ = transmat[appliance] 
    model[appliance].means_ = means[appliance]
    model[appliance].covars_ = covars[appliance]

    
new_model = OrderedDict()
for appliance in model:
    startprob_new, means_new, covars_new, transmat_new = sort_learnt_parameters(
                startprob[appliance], means[appliance], 
        covars[appliance], transmat[appliance])
    
    new_model[appliance] = GaussianHMM(n_components=start_prob_new.size, covariance_type="full")
    new_model[appliance].startprob_ = startprob_new
    new_model[appliance].transmat_ = transmat_new
    new_model[appliance].means_ = means_new
    new_model[appliance].covars_ = covars_new


combined_model = create_combined_hmm(new_model)


# Testing data and predicting power of each appliance 
test_power = data['powerTest']
test_power = test_power.split(',')


for i in range(len(test_power)):
    test_power[i] = float(test_power[i])
    
    
test_power = sp.signal.medfilt(np.array(test_power))
length = len(test_power)
[predicted_power, predicted_states] = disaggregate(combined_model, new_model, test_power)


predicted_total = np.zeros(length)
for appliance in predicted_power: 
    predicted_power[appliance] = predicted_power[appliance]
    predicted_total += predicted_power[appliance]
    
buff = "\"power\" : {"
for appliance in predicted_power:
    buff += "\power" + appliance +"\" :" + str(predicted_states[appliance].tolist()) + ","
    
    
buff = buff[0:len(buff)-1]
buff += "} }"


print(buff)


NameError: name 'model_appliance' is not defined

In [ ]:
import sys
import json
import itertools
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from scipy import signal 
from copy import deepcopy
from hmmlearn.hmm import GaussianHMM
from collections import OrderedDict

""" --------------------- Fonctions ---------------------------"""

def return_sorting_mapping(means):
    means_copy = deepcopy(means)
    means_copy = np.sort(means_copy, axis=0)
    
    # Trouver le mapping
    mapping = {}
    for i, val in enumerate(means_copy):
        mapping[i] = np.where(val == means)[0][0]
    return mapping

def sort_startprob(mapping, startprob):
    num_elements = len(startprob)
    new_startprob = np.zeros(num_elements)
    for i in range(len(startprob)):
        new_startprob[i] = startprob[mapping[i]]
    return new_startprob

def sort_covars(mapping, covars):
    new_covars = np.zeros_like(covars)
    for i in range(len(covars)):
        new_covars[i] = covars[mapping[i]]
    return new_covars

def sort_transition_matrix(mapping, A):
    num_elements = len(A)
    A_new = np.zeros((num_elements, num_elements))
    for i in range(num_elements):
        for j in range(num_elements):
            A_new[i, j] = A[mapping[i], A[mapping[j]]]
    return A_new

def sort_learnt_parameters(startprob, means, covars, transmat):
    mapping = return_sorting_mapping(means)
    means_new = np.sort(means, axis=0)
    startprob_new = sort_startprob(mapping, startprob)
    covars_new = sort_covars(mapping, covars)
    transmat_new = sort_transition_matrix(mapping, transmat)
    assert np.shape(means_new) == np.shape(means)
    assert np.shape(startprob_new) == np.shape(startprob)
    assert np.shape(transmat_new) == np.shape(transmat)
    
    return [startprob_new, means_new, covars_new, transmat_new]

def compute_A_fhmm(list_A):
    result = list_A[0]
    for i in range(len(list_A) - 1):
        result = np.kron(result, list_A[i + 1])
    return result

def compute_pi_fhmm(list_pi):
    result = list_pi[0]
    for i in range(len(list_pi) - 1):
        result = np.kron(result, list_pi[i + 1])
    return result

def compute_means_fhmm(list_means):
    states_combination = list(itertools.product(*list_means))
    num_combinations = len(states_combination)
    means_stacked = np.array([sum(x) for x in states_combination])
    means = np.reshape(means_stacked, (num_combinations, 1))
    cov = np.tile(5 * np.identity(1), (num_combinations, 1, 1))

    return [means, cov]

def create_combined_fhmm(model):
    list_pi = [model[appliance].startprob_ for appliance in model]
    list_A = [model[appliance].transmat_ for appliance in model]
    list_means = [model[appliance].means_.flatten().tolist() for appliance in model]

    pi_combined = compute_pi_fhmm(list_pi)
    A_combined = compute_A_fhmm(list_A)
    [means_combined, cov_combined] = compute_means_fhmm(list_means)

    combined_model = GaussianHMM(n_components=len(pi_combined), covariance_type='full')
    combined_model.startprob_ = pi_combined
    combined_model.transmat_ = A_combined
    combined_model.means_ = means_combined
    combined_model.covars_ = cov_combined

    return combined_model

def decode_hmm(length_sequence, centroids, appliance_list, states):
    hmm_states = OrderedDict()
    hmm_power = OrderedDict()
    total_num_combinations = 1
    
    for appliance in appliance_list:
        total_num_combinations *= len(centroids[appliance])
    
    for appliance in appliance_list:
        hmm_states[appliance] = np.zeros(length_sequence, dtype=int)
        hmm_power[appliance] = np.zeros(length_sequence)
    
    for i in range(length_sequence):
        factor = total_num_combinations
        for appliance in appliance_list:
            factor = factor // len(centroids[appliance])
            temp = int(states[i]) // factor
            hmm_states[appliance][i] = temp % len(centroids[appliance])
            hmm_power[appliance][i] = centroids[appliance][hmm_states[appliance][i]]
    
    return [hmm_states, hmm_power]

def disaggregate(combined_model, model, test_power):
    length = len(test_power)
    temp = test_power.reshape(length, 1)
    states = combined_model.predict(temp)
    
    means = OrderedDict()
    for appliance in model:
        means[appliance] = model[appliance].means_.flatten().tolist()
        means[appliance].sort()
        
    [decoded_states, decoded_power] = decode_hmm(len(states), means, means.keys(), states)
    
    return [decoded_power, decoded_states]

# Lecture des données depuis le fichier CSV
data = pd.read_csv("./AMPds/test_ampds2.csv")

# Modèle d'appareils
state_appliances = OrderedDict()
state_appliances['BME'] = 2
state_appliances['CDE'] = 3
state_appliances['FGE'] = 3
state_appliances['HPE'] = 2
state_appliances['TVE'] = 1

startprob = OrderedDict()
transmat = OrderedDict()
means = OrderedDict()
covars = OrderedDict()
model = OrderedDict()

# Chargement des paramètres appris pour chaque appareil
model_appliance = {
    'BME': {'startprob': [0.6, 0.4], 'transmat': [[0.7, 0.3], [0.4, 0.6]], 'means': [[10], [20]], 'covars': [[[1]], [[1]]]},
    'CDE': {'startprob': [0.5, 0.3, 0.2], 'transmat': [[0.6, 0.2, 0.2], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]], 'means': [[30], [40], [50]], 'covars': [[[1]], [[1]], [[1]]]},
    'FGE': {'startprob': [0.4, 0.4, 0.2], 'transmat': [[0.5, 0.3, 0.2], [0.3, 0.4, 0.3], [0.2, 0.3, 0.5]], 'means': [[60], [70], [80]], 'covars': [[[1]], [[1]], [[1]]]},
    'HPE': {'startprob': [0.7, 0.3], 'transmat': [[0.6, 0.4], [0.3, 0.7]], 'means': [[90], [100]], 'covars': [[[1]], [[1]]]},
    'TVE': {'startprob': [1.0], 'transmat': [[1.0]], 'means': [[110]], 'covars': [[[1]]]}
}

for appliance in model_appliance:
    startprob[appliance] = np.array(model_appliance[appliance]['startprob'])
    transmat[appliance] = np.array(model_appliance[appliance]['transmat'])
    means[appliance] = np.array(model_appliance[appliance]['means'])
    covars[appliance] = np.array(model_appliance[appliance]['covars'])

for appliance in model_appliance:
    model[appliance] = GaussianHMM(n_components=state_appliances[appliance], covariance_type='full')
    model[appliance].startprob_ = startprob[appliance]
    model[appliance].transmat_ = transmat[appliance]
    model[appliance].means_ = means[appliance]
    model[appliance].covars_ = covars[appliance]

# Trier les paramètres appris
new_model = OrderedDict()
for appliance in model:
    startprob_new, means_new, covars_new, transmat_new = sort_learnt_parameters(
        startprob[appliance], means[appliance], covars[appliance], transmat[appliance]
    )
    new_model[appliance] = GaussianHMM(n_components=startprob_new.size, covariance_type="full")
    new_model[appliance].startprob_ = startprob_new
    new_model[appliance].transmat_ = transmat_new
    new_model[appliance].means_ = means_new
    new_model[appliance].covars_ = covars_new

combined_model = create_combined_fhmm(new_model)

# Données de test et prédiction de la puissance de chaque appareil
test_power = data['Total'].values
test_power = sp.signal.medfilt(np.array(test_power))
length = len(test_power)
[predicted_power, predicted_states] = disaggregate(combined_model, new_model, test_power)

predicted_total = np.zeros(length)
for appliance in predicted_power:
    predicted_total += predicted_power[appliance]

buff = "\"power\" : {"
for appliance in predicted_power:
    buff += f"\"power{appliance}\" : {predicted_states[appliance].tolist()},"
    
buff = buff[:-1]
buff += "}"

print(buff)


In [14]:
import sys
import json
import itertools
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from scipy import signal 
from copy import deepcopy
from hmmlearn.hmm import GaussianHMM
from collections import OrderedDict

""" --------------------- Fonctions ---------------------------"""

def return_sorting_mapping(means):
    means_copy = deepcopy(means)
    means_copy = np.sort(means_copy, axis=0)
    
    # Trouver le mapping
    mapping = {}
    for i, val in enumerate(means_copy):
        mapping[i] = np.where(val == means)[0][0]
    return mapping

def sort_startprob(mapping, startprob):
    num_elements = len(startprob)
    new_startprob = np.zeros(num_elements)
    for i in range(len(startprob)):
        new_startprob[i] = startprob[mapping[i]]
    return new_startprob

def sort_covars(mapping, covars):
    new_covars = np.zeros_like(covars)
    for i in range(len(covars)):
        new_covars[i] = covars[mapping[i]]
    return new_covars

def sort_transition_matrix(mapping, A):
    num_elements = len(A)
    A_new = np.zeros((num_elements, num_elements))
    for i in range(num_elements):
        for j in range(num_elements):
            A_new[i, j] = A[mapping[i], mapping[j]]
    return A_new

def sort_learnt_parameters(startprob, means, covars, transmat):
    mapping = return_sorting_mapping(means)
    means_new = np.sort(means, axis=0)
    startprob_new = sort_startprob(mapping, startprob)
    covars_new = sort_covars(mapping, covars)
    transmat_new = sort_transition_matrix(mapping, transmat)
    assert np.shape(means_new) == np.shape(means)
    assert np.shape(startprob_new) == np.shape(startprob)
    assert np.shape(transmat_new) == np.shape(transmat)
    
    return [startprob_new, means_new, covars_new, transmat_new]

def compute_A_fhmm(list_A):
    result = list_A[0]
    for i in range(len(list_A) - 1):
        result = np.kron(result, list_A[i + 1])
    return result

def compute_pi_fhmm(list_pi):
    result = list_pi[0]
    for i in range(len(list_pi) - 1):
        result = np.kron(result, list_pi[i + 1])
    return result

def compute_means_fhmm(list_means):
    states_combination = list(itertools.product(*list_means))
    num_combinations = len(states_combination)
    means_stacked = np.array([sum(x) for x in states_combination])
    means = np.reshape(means_stacked, (num_combinations, 1))
    cov = np.tile(5 * np.identity(1), (num_combinations, 1, 1))

    return [means, cov]

def create_combined_fhmm(model):
    list_pi = [model[appliance].startprob_ for appliance in model]
    list_A = [model[appliance].transmat_ for appliance in model]
    list_means = [model[appliance].means_.flatten().tolist() for appliance in model]

    pi_combined = compute_pi_fhmm(list_pi)
    A_combined = compute_A_fhmm(list_A)
    [means_combined, cov_combined] = compute_means_fhmm(list_means)

    combined_model = GaussianHMM(n_components=len(pi_combined), covariance_type='full')
    combined_model.startprob_ = pi_combined
    combined_model.transmat_ = A_combined
    combined_model.means_ = means_combined
    combined_model.covars_ = cov_combined

    return combined_model

def decode_hmm(length_sequence, centroids, appliance_list, states):
    hmm_states = OrderedDict()
    hmm_power = OrderedDict()
    total_num_combinations = 1
    
    for appliance in appliance_list:
        total_num_combinations *= len(centroids[appliance])
    
    for appliance in appliance_list:
        hmm_states[appliance] = np.zeros(length_sequence, dtype=int)
        hmm_power[appliance] = np.zeros(length_sequence)
    
    for i in range(length_sequence):
        factor = total_num_combinations
        for appliance in appliance_list:
            factor = factor // len(centroids[appliance])
            temp = int(states[i]) // factor
            hmm_states[appliance][i] = temp % len(centroids[appliance])
            hmm_power[appliance][i] = centroids[appliance][hmm_states[appliance][i]]
    
    return [hmm_states, hmm_power]

def disaggregate(combined_model, model, test_power):
    length = len(test_power)
    temp = test_power.reshape(length, 1)
    states = combined_model.predict(temp)
    
    means = OrderedDict()
    for appliance in model:
        means[appliance] = model[appliance].means_.flatten().tolist()
        means[appliance].sort()
        
    [decoded_states, decoded_power] = decode_hmm(len(states), means, means.keys(), states)
    
    return [decoded_power, decoded_states]

# Lecture des données depuis le fichier CSV
data = pd.read_csv("./AMPds/test_ampds2.csv")

# Modèle d'appareils avec les paramètres fournis
model_appliance = data = pd.read_csv("ampds.csv")
{
    'BME': {
        'startprob': [0.5, 0.5],
        'transmat': [[0.987137989930789, 0.012862010069210927], [0.0524049637956063, 0.9475950362043937]],
        'means': [[7.000000000000008], [291.96352025124975]],
        'covars': [[[6.183605062374777e-07]], [[14571.356085794527]]]
    },
    'DWE': {
        'startprob': [0.33333333, 0.33333333, 0.33333333],
        'transmat': [[0.9997435239426238, 0.0, 0.0002564760573762622], [0.0, 0.920863309267975, 0.079136690732025], [0.012953366166343568, 0.05699480282809405, 0.9300518310055623]],
        'means': [[0.0], [752.9352517985611], [31.261649284048147]],
        'covars': [[[5.129257311987589e-07]], [[223.8879299725903]], [[2679.7264214259744]]]
    },
    'FGE': {
        'startprob': [0.33333333, 0.33333333, 0.33333333],
        'transmat': [[0.9594675441243959, 0.040233323359876796, 0.00029913251572728517], [0.07994056462468897, 0.920059435375311, 0.0], [0.07017543856990009, 0.0, 0.9298245614301]],
        'means': [[0.9092948478277125], [129.192867756315], [439.7894736842105]],
        'covars': [[[5.340386199836612]], [[49.641258194995316]], [[64.58743305632159]]]
    },
    'HPE': {
        'startprob': [0.5, 0.5],
        'transmat': [[0.9905764288293913, 0.009423571170608607], [0.04034582132815857, 0.9596541786718414]],
        'means': [[30.92222970078933], [1768.9384333246005]],
        'covars': [[[184.13523612248068]], [[12688.073497991523]]]
    },
    'UTE': {
        'startprob': [1.0],
        'transmat': [[1.0]],
        'means': [[8.986507936507936]],
        'covars': [[[0.016088797556053448]]]
    }
}

state_appliances = {
    'BME': 2,
    'DWE': 3,
    'FGE': 3,
    'HPE': 2,
    'UTE': 1
}

startprob = OrderedDict()
transmat = OrderedDict()
means = OrderedDict()
covars = OrderedDict()
model = OrderedDict()

for appliance in model_appliance:
    startprob[appliance] = np.array(model_appliance[appliance]['startprob'])
    transmat[appliance] = np.array(model_appliance[appliance]['transmat'])
    means[appliance] = np.array(model_appliance[appliance]['means'])
    covars[appliance] = np.array(model_appliance[appliance]['covars'])

for appliance in model_appliance:
    model[appliance] = GaussianHMM(n_components=state_appliances[appliance], covariance_type='full')
    model[appliance].startprob_ = startprob[appliance]
    model[appliance].transmat_ = transmat[appliance]
    model[appliance].means_ = means[appliance]
    model[appliance].covars_ = covars[appliance]

new_model = OrderedDict()
for appliance in model:
    startprob_new, means_new, covars_new, transmat_new = sort_learnt_parameters(
        startprob[appliance], means[appliance], covars[appliance], transmat[appliance]
    )
    new_model[appliance] = GaussianHMM(n_components=startprob_new.size, covariance_type="full")
    new_model[appliance].startprob_ = startprob_new
    new_model[appliance].transmat_ = transmat_new
    new_model[appliance].means_ = means_new
    new_model[appliance].covars_ = covars_new

combined_model = create_combined_fhmm(new_model)

# Données de test et prédiction de la puissance de chaque appareil
test_power = data['Total'].values
test_power = sp.signal.medfilt(np.array(test_power))
length = len(test_power)
[predicted_power, predicted_states] = disaggregate(combined_model, new_model, test_power)

# Calcul de la puissance totale prédite
predicted_total = np.zeros(length)
for appliance in predicted_power:
    predicted_total += predicted_power[appliance]

# Préparation de la sortie des résultats
buff = "\"power\" : {"
for appliance in predicted_power:
    buff += f"\"power{appliance}\" : {predicted_states[appliance].tolist()},"
buff = buff[:-1]
buff += "}"

print(buff)


"power" : {"powerBME" : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [16]:
import sys
import json
import itertools
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from scipy import signal 
from copy import deepcopy
from hmmlearn.hmm import GaussianHMM
from collections import OrderedDict

""" --------------------- Fonctions ---------------------------"""

def return_sorting_mapping(means):
    means_copy = deepcopy(means)
    means_copy = np.sort(means_copy, axis=0)
    
    # Trouver le mapping
    mapping = {}
    for i, val in enumerate(means_copy):
        mapping[i] = np.where(val == means)[0][0]
    return mapping

def sort_startprob(mapping, startprob):
    num_elements = len(startprob)
    new_startprob = np.zeros(num_elements)
    for i in range(len(startprob)):
        new_startprob[i] = startprob[mapping[i]]
    return new_startprob

def sort_covars(mapping, covars):
    new_covars = np.zeros_like(covars)
    for i in range(len(covars)):
        new_covars[i] = covars[mapping[i]]
    return new_covars

def sort_transition_matrix(mapping, A):
    num_elements = len(A)
    A_new = np.zeros((num_elements, num_elements))
    for i in range(num_elements):
        for j in range(num_elements):
            A_new[i, j] = A[mapping[i], A[mapping[j]]]
    return A_new

def sort_learnt_parameters(startprob, means, covars, transmat):
    mapping = return_sorting_mapping(means)
    means_new = np.sort(means, axis=0)
    startprob_new = sort_startprob(mapping, startprob)
    covars_new = sort_covars(mapping, covars)
    transmat_new = sort_transition_matrix(mapping, transmat)
    assert np.shape(means_new) == np.shape(means)
    assert np.shape(startprob_new) == np.shape(startprob)
    assert np.shape(transmat_new) == np.shape(transmat)
    
    return [startprob_new, means_new, covars_new, transmat_new]

def compute_A_fhmm(list_A):
    result = list_A[0]
    for i in range(len(list_A) - 1):
        result = np.kron(result, list_A[i + 1])
    return result

def compute_pi_fhmm(list_pi):
    result = list_pi[0]
    for i in range(len(list_pi) - 1):
        result = np.kron(result, list_pi[i + 1])
    return result

def compute_means_fhmm(list_means):
    states_combination = list(itertools.product(*list_means))
    num_combinations = len(states_combination)
    means_stacked = np.array([sum(x) for x in states_combination])
    means = np.reshape(means_stacked, (num_combinations, 1))
    cov = np.tile(5 * np.identity(1), (num_combinations, 1, 1))

    return [means, cov]

def create_combined_fhmm(model):
    list_pi = [model[appliance].startprob_ for appliance in model]
    list_A = [model[appliance].transmat_ for appliance in model]
    list_means = [model[appliance].means_.flatten().tolist() for appliance in model]

    pi_combined = compute_pi_fhmm(list_pi)
    A_combined = compute_A_fhmm(list_A)
    [means_combined, cov_combined] = compute_means_fhmm(list_means)

    combined_model = GaussianHMM(n_components=len(pi_combined), covariance_type='full')
    combined_model.startprob_ = pi_combined
    combined_model.transmat_ = A_combined
    combined_model.means_ = means_combined
    combined_model.covars_ = cov_combined

    return combined_model

def decode_hmm(length_sequence, centroids, appliance_list, states):
    hmm_states = OrderedDict()
    hmm_power = OrderedDict()
    total_num_combinations = 1
    
    for appliance in appliance_list:
        total_num_combinations *= len(centroids[appliance])
    
    for appliance in appliance_list:
        hmm_states[appliance] = np.zeros(length_sequence, dtype=int)
        hmm_power[appliance] = np.zeros(length_sequence)
    
    for i in range(length_sequence):
        factor = total_num_combinations
        for appliance in appliance_list:
            factor = factor // len(centroids[appliance])
            temp = int(states[i]) // factor
            hmm_states[appliance][i] = temp % len(centroids[appliance])
            hmm_power[appliance][i] = centroids[appliance][hmm_states[appliance][i]]
    
    return [hmm_states, hmm_power]

def disaggregate(combined_model, model, test_power):
    length = len(test_power)
    temp = test_power.reshape(length, 1)
    states = combined_model.predict(temp)
    
    means = OrderedDict()
    for appliance in model:
        means[appliance] = model[appliance].means_.flatten().tolist()
        means[appliance].sort()
        
    [decoded_states, decoded_power] = decode_hmm(len(states), means, means.keys(), states)
    
    return [decoded_power, decoded_states]

# Lecture des données depuis le fichier CSV
data = pd.read_csv("./AMPds/test_ampds2.csv")

# Modèle d'appareils
state_appliances = OrderedDict()
state_appliances['BME'] = 2
state_appliances['CDE'] = 3
state_appliances['FGE'] = 3
state_appliances['HPE'] = 2
state_appliances['TVE'] = 1

startprob = OrderedDict()
transmat = OrderedDict()
means = OrderedDict()
covars = OrderedDict()
model = OrderedDict()

# Chargement des paramètres appris pour chaque appareil à partir du fichier trained_models.txt
with open("trained_models.txt", "r") as file:
    trained_models = json.load(file)

for appliance, params in trained_models.items():
    startprob[appliance] = params["startprob"]
    transmat[appliance] = params["transmat"]
    means[appliance] = params["means"]
    covars[appliance] = params["covars"]

for appliance in trained_models:
    model[appliance] = GaussianHMM(n_components=state_appliances[appliance], covariance_type='full')
    model[appliance].startprob_ = startprob[appliance]
    model[appliance].transmat_ = transmat[appliance]
    model[appliance].means_ = means[appliance]
    model[appliance].covars_ = covars[appliance]

# Trier les paramètres appris
new_model = OrderedDict()
for appliance in model:
    startprob_new, means_new, covars_new, transmat_new = sort_learnt_parameters(
        startprob[appliance], means[appliance], covars[appliance], transmat[appliance]
    )
    new_model[appliance] = GaussianHMM(n_components=startprob_new.size, covariance_type="full")
    new_model[appliance].startprob_ = startprob_new
    new_model[appliance].transmat_ = transmat_new
    new_model[appliance].means_ = means_new
    new_model[appliance].covars_ = covars_new

combined_model = create_combined_fhmm(new_model)

# Données de test et prédiction de la puissance de chaque appareil
test_power = data['Total'].values
test_power = sp.signal.medfilt(np.array(test_power))
length = len(test_power)
[predicted_power, predicted_states] = disaggregate(combined_model, new_model, test_power)

# Création du graphique pour la puissance prédite de chaque appareil
plt.figure(figsize=(10, 6))
for appliance, power in predicted_power.items():
    plt.plot(power, label=appliance)

plt.xlabel('Temps')
plt.ylabel('Puissance (W)')
plt.title('Puissance prédite de chaque appareil')
plt.legend()
plt.grid(True)
plt.show()

# Création du graphique pour la puissance totale prédite
plt.figure(figsize=(10, 6))
plt.plot(test_power, label='Puissance totale réelle', color='black', linestyle='--')
plt.plot(np.sum(list(predicted_power.values()), axis=0), label='Puissance totale prédite', color='red')

plt.xlabel('Temps')
plt.ylabel('Puissance (W)')
plt.title('Puissance totale réelle vs Puissance totale prédite')
plt.legend()
plt.grid(True)
plt.show()



JSONDecodeError: Expecting ',' delimiter: line 1 column 30 (char 29)

In [ ]:
# Trier les paramètres appris
new_model = OrderedDict()
for appliance in model:
    startprob_new, means_new, covars_new, transmat_new = sort_learnt_parameters(
        startprob[appliance], means[appliance], covars[appliance], transmat[appliance]
    )
    new_model[appliance] = GaussianHMM(n_components=startprob_new.size, covariance_type="full")
    new_model[appliance].startprob_ = startprob_new
    new_model[appliance].transmat_ = transmat_new
    new_model[appliance].means_ = means_new
    new_model[appliance].covars_ = covars_new

combined_model = create_combined_fhmm(new_model)

# Données de test et prédiction de la puissance de chaque appareil
test_power = data['Total'].values
test_power = sp.signal.medfilt(np.array(test_power))
length = len(test_power)
[predicted_power, predicted_states] = disaggregate(combined_model, new_model, test_power)

# Création du graphique pour la puissance prédite de chaque appareil
plt.figure(figsize=(10, 6))
for appliance, power in predicted_power.items():
    plt.plot(power, label=appliance)

plt.xlabel('Temps')
plt.ylabel('Puissance (W)')
plt.title('Puissance prédite de chaque appareil')
plt.legend()
plt.grid(True)
plt.show()

# Création du graphique pour la puissance totale prédite
plt.figure(figsize=(10, 6))
plt.plot(test_power, label='Puissance totale réelle', color='black', linestyle='--')
plt.plot(np.sum(list(predicted_power.values()), axis=0), label='Puissance totale prédite', color='red')

plt.xlabel('Temps')
plt.ylabel('Puissance (W)')
plt.title('Puissance totale réelle vs Puissance totale prédite')
plt.legend()
plt.grid(True)
plt.show()
